In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# cd /content/drive/My Drive/University/SEM7/CS4622_Machine_Learning/Project_1/Fair_Classification

# Imports

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import calendar
import warnings
from math import sin, cos, sqrt, atan2, radians,asin
import math

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
# import lightgbm as lgb
# import xgboost as xgb
from sklearn.metrics import roc_auc_score
# from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
warnings.filterwarnings('ignore')
# from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score,f1_score


# Load Data

In [4]:
# train=pd.read_csv("/content/drive/My Drive/University/SEM7/CS4622_Machine_Learning/Project_1/Fair_Classification/features/Train_FeatureEngineering_4_without_droping_any_feature.csv")
# test=pd.read_csv("/content/drive/My Drive/University/SEM7/CS4622_Machine_Learning/Project_1/Fair_Classification/features/Test_FeatureEngineering_4.csv")
# for other account
train=pd.read_csv("features/8_Train_FeatureEngineering_fill_null_with_mean.csv")
test=pd.read_csv("features/8_Test_FeatureEngineering_fill_null_with_mean.csv")

print("Shape of Training Data",train.shape)
print("Shape of Testing Data", test.shape)


Shape of Training Data (17176, 25)
Shape of Testing Data (8576, 24)


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17176 entries, 0 to 17175
Data columns (total 25 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   tripid                         17176 non-null  int64  
 1   additional_fare                17176 non-null  float64
 2   duration                       17176 non-null  float64
 3   meter_waiting                  17176 non-null  float64
 4   meter_waiting_fare             17176 non-null  float64
 5   meter_waiting_till_pickup      17176 non-null  float64
 6   pickup_time                    17176 non-null  object 
 7   drop_time                      17176 non-null  object 
 8   pick_lat                       17176 non-null  float64
 9   pick_lon                       17176 non-null  float64
 10  drop_lat                       17176 non-null  float64
 11  drop_lon                       17176 non-null  float64
 12  fare                           17176 non-null 

In [6]:
test.columns

Index(['tripid', 'additional_fare', 'duration', 'meter_waiting',
       'meter_waiting_fare', 'meter_waiting_till_pickup', 'pickup_time',
       'drop_time', 'pick_lat', 'pick_lon', 'drop_lat', 'drop_lon', 'fare',
       'additional_fare_is_outlier', 'meter_waiting_fare_is_outlier',
       'fare_is_outlier', 'pickup_date', 'pickup_day', 'pickup_hour',
       'pickup_day_of_week', 'pickup_month', 'pickup_year', 'is_outlier_loc',
       'trip_distance'],
      dtype='object')

# Data_Preprocessing

In [7]:
# train = train.fillna(train.mean())
# test = test.fillna(test.mean())

In [8]:
def is_time_ok(duration,meter_waiting):
    if (duration>meter_waiting):
        return 1
    else:
        return 0
    
def speed(distance,duration):
    if (duration<=0):
        return np.nan
    else:
        speed = distance/duration
        return speed
    

In [9]:
train['is_time_ok'] = train.apply(lambda row:is_time_ok(row['duration'],row['meter_waiting']),axis=1)
test['is_time_ok'] = test.apply(lambda row:is_time_ok(row['duration'],row['meter_waiting']),axis=1)
train['speed'] = train.apply(lambda row:speed(row['trip_distance'],row['duration']),axis=1)
test['speed'] = test.apply(lambda row:speed(row['trip_distance'],row['duration']),axis=1)

In [10]:
train['speed'].isna().sum() 

2

In [11]:
train.mean()

tripid                           2.014549e+08
additional_fare                  1.371965e+01
duration                         1.702858e+03
meter_waiting                    6.290742e+02
meter_waiting_fare               3.205767e+01
meter_waiting_till_pickup        1.124668e+02
pick_lat                         6.965499e+00
pick_lon                         7.999839e+01
drop_lat                         6.968587e+00
drop_lon                         7.999753e+01
fare                             3.126555e+02
label                            9.021309e-01
additional_fare_is_outlier       1.187704e-02
meter_waiting_fare_is_outlier    1.467163e-02
fare_is_outlier                  1.309967e-02
pickup_day                       1.590359e+01
pickup_hour                      1.381754e+01
pickup_day_of_week               3.118945e+00
pickup_month                     7.955170e+00
pickup_year                      2.019339e+03
is_outlier_loc                   4.657662e-04
trip_distance                    3

In [12]:
# drop_columns_train=['tripid','pickup_time','drop_time','pick_lat','pick_lon','drop_lat','drop_lon','pickup_date','pickup_latitude_round3','pickup_longitude_round3','dropoff_latitude_round3','dropoff_longitude_round3']
# drop_columns_test=['tripid','pickup_time','drop_time','pick_lat','pick_lon','drop_lat','drop_lon','pickup_date','pickup_latitude_round3','pickup_longitude_round3','dropoff_latitude_round3','dropoff_longitude_round3','is_outlier_loc']

drop_columns_train=['tripid','pickup_time','drop_time','pickup_day','pickup_month','pickup_year']
drop_columns_test=['tripid','pickup_time','drop_time','pickup_day','pickup_month','pickup_year']

In [13]:
train_1=train.drop(drop_columns_train,axis=1)
test_1=test.drop(drop_columns_test,axis=1)
print("Shape of Training Data after dropping columns",train_1.shape)
print("Shape of Testing Data after dropping columns",test_1.shape)

Shape of Training Data after dropping columns (17176, 21)
Shape of Testing Data after dropping columns (8576, 20)


In [14]:
# def encodeDays(day_of_week):
#   day_dict={'Sunday':0,'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6}
#   return day_dict[day_of_week]
# def encodeLabel(label):
#   label_dict={'correct':1,'incorrect':0}
#   return label_dict[label]
# def clean_data(data,isTrain):
#   data['pickup_day_of_week']=data['pickup_day_of_week'].apply(lambda x:encodeDays(x))
#   if(isTrain):
#     data['label']=data['label'].apply(lambda x:encodeLabel(x))
#   return data

# train_1=clean_data(train_1,True)
# test_1=clean_data(test_1,False)
# print("Shape of Training Data after cleaning ",train_1.shape)
# print("Shape of Testing Data after cleaning", test_1.shape)

In [15]:
test_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8576 entries, 0 to 8575
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   additional_fare                8576 non-null   float64
 1   duration                       8576 non-null   int64  
 2   meter_waiting                  8576 non-null   int64  
 3   meter_waiting_fare             8576 non-null   float64
 4   meter_waiting_till_pickup      8576 non-null   int64  
 5   pick_lat                       8576 non-null   float64
 6   pick_lon                       8576 non-null   float64
 7   drop_lat                       8576 non-null   float64
 8   drop_lon                       8576 non-null   float64
 9   fare                           8576 non-null   float64
 10  additional_fare_is_outlier     8576 non-null   int64  
 11  meter_waiting_fare_is_outlier  8576 non-null   int64  
 12  fare_is_outlier                8576 non-null   i

In [16]:
columns_titles = ['additional_fare','duration','meter_waiting','meter_waiting_fare','meter_waiting_till_pickup','pick_lat','pick_lon','drop_lat','drop_lon','fare','additional_fare_is_outlier','meter_waiting_fare_is_outlier','fare_is_outlier','pickup_hour','pickup_day_of_week','is_outlier_loc','trip_distance','is_time_ok','speed','label']
train_1=train_1.reindex(columns=columns_titles)
train_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17176 entries, 0 to 17175
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   additional_fare                17176 non-null  float64
 1   duration                       17176 non-null  float64
 2   meter_waiting                  17176 non-null  float64
 3   meter_waiting_fare             17176 non-null  float64
 4   meter_waiting_till_pickup      17176 non-null  float64
 5   pick_lat                       17176 non-null  float64
 6   pick_lon                       17176 non-null  float64
 7   drop_lat                       17176 non-null  float64
 8   drop_lon                       17176 non-null  float64
 9   fare                           17176 non-null  float64
 10  additional_fare_is_outlier     17176 non-null  int64  
 11  meter_waiting_fare_is_outlier  17176 non-null  int64  
 12  fare_is_outlier                17176 non-null 

In [17]:
categorical_var = ['additional_fare_is_outlier','meter_waiting_fare_is_outlier','fare_is_outlier','pickup_hour','pickup_day_of_week','is_outlier_loc','is_time_ok'] 
categorical_var_indices = [train_1.columns.get_loc(c) for c in categorical_var if c in train_1]
print('\nCategorical Variables indices : ',categorical_var_indices)


Categorical Variables indices :  [10, 11, 12, 13, 14, 15, 17]


In [18]:
train_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17176 entries, 0 to 17175
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   additional_fare                17176 non-null  float64
 1   duration                       17176 non-null  float64
 2   meter_waiting                  17176 non-null  float64
 3   meter_waiting_fare             17176 non-null  float64
 4   meter_waiting_till_pickup      17176 non-null  float64
 5   pick_lat                       17176 non-null  float64
 6   pick_lon                       17176 non-null  float64
 7   drop_lat                       17176 non-null  float64
 8   drop_lon                       17176 non-null  float64
 9   fare                           17176 non-null  float64
 10  additional_fare_is_outlier     17176 non-null  int64  
 11  meter_waiting_fare_is_outlier  17176 non-null  int64  
 12  fare_is_outlier                17176 non-null 

In [19]:
# train_1=pd.get_dummies(train_1)
# test_1=pd.get_dummies(test_1)

# print("Shape of Training Data after One Hot Encoding",train_1.shape)
# print("Shape of Testing Data after One Hot Encoding",test_1.shape)

In [20]:
# print('Old size: %d' % len(train))
# train_1 = train_1.dropna()
# print('New size: %d' % len(train))

In [21]:
X=train_1.drop(['label'],axis=1)
y=train_1['label']

In [22]:
#split data into train and validation data
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)
print("Number of records in training data ",X_train.shape[0])
print("Number of records in validation data ",X_test.shape[0])

Number of records in training data  13740
Number of records in validation data  3436


# Classifiers

## XGBoost CLassifier

In [23]:
# model = XGBClassifier()
# model.fit(X_train, y_train)

## Linear Regression Classifier

In [24]:
# model = LinearRegression()
# model.fit(X_train,y_train)
# # y_pred=model.predict(X_test)
# # lm_rmse=np.sqrt(mean_squared_error(y_pred, y_test))
# # print("RMSE for Linear Regression is ",lm_rmse)


## LGBM Classifier

In [25]:
# estimator = lgb.LGBMClassifier(learning_rate = 0.125, metric = 'l1', 
#                         n_estimators = 20, num_leaves = 38)


# param_grid = {
#     'n_estimators': [x for x in range(24,40,2)],
#     'learning_rate': [0.10, 0.125, 0.15, 0.175, 0.2]}
# gridsearch = GridSearchCV(estimator, param_grid)

# gridsearch.fit(X_train, y_train,
#         eval_set = [(X_test, y_test)],
#         eval_metric = ['auc', 'binary_logloss'],
#         early_stopping_rounds = 5)

In [26]:
# gbm = lgb.LGBMClassifier(boosting_type='gbdt', class_weight=None,
#                                       colsample_bytree=1.0,
#                                       importance_type='split',
#                                       learning_rate=0.125, max_depth=-1,
#                                       metric='l1', min_child_samples=20,
#                                       min_child_weight=0.001,
#                                       min_split_gain=0.0, n_estimators=20,
#                                       n_jobs=-1, num_leaves=38, objective=None,
#                                       random_state=None, reg_alpha=0.0,
#                                       reg_lambda=0.0, silent=True,
#                                       subsample=1.0, subsample_for_bin=200000,
#                                       subsample_freq=0)


# gbm.fit(X, y,
#         eval_set=[(X_test, y_test)],
#         eval_metric=['auc', 'binary_logloss'],
# early_stopping_rounds=5)

In [27]:
# ax = lgb.plot_importance(gbm, height = 0.4, 
#                          max_num_features = 50, 
#                          xlim = (0,100), ylim = (0,23), 
#                          figsize = (10,6))
# plt.show()

In [28]:
# sorted(gbm.feature_importances_,reverse=True)

In [29]:

# # The code below aims to drop  to keep the features that are included in the most important features. 
# temp = 0 
# total = sum(gbm.feature_importances_)
# for feature in sorted(gbm.feature_importances_, reverse=True):
#     temp+=feature
#     print(feature)
#     if temp/total >= 0.85:
#         print(feature,temp/total) # stop when we 
#         break

In [30]:
# #The above means let go of all variables after PAY_AMT_5
# y_pred_prob = gbm.predict_proba(test_1)[:, 1]
# # auc_roc_0 = str(roc_auc_score(y_test, y_pred_prob)) # store AUC score without dimensionality reduction
# # print('AUC without dimensionality reduction: \n' + auc_roc_0)

In [31]:
X

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare,additional_fare_is_outlier,meter_waiting_fare_is_outlier,fare_is_outlier,pickup_hour,pickup_day_of_week,is_outlier_loc,trip_distance,is_time_ok,speed
0,10.500000,834.000000,56.000000,0.000000,64.000000,6.86252,79.8993,6.90330,79.8783,270.32,0,0,0,0,5,0,3.164501,1,0.003794
1,10.500000,791.000000,47.000000,0.000000,134.000000,6.88589,79.8984,6.91373,79.8923,197.85,0,0,0,0,5,0,1.968540,1,0.002489
2,10.500000,1087.000000,80.000000,0.000000,61.000000,6.90839,79.8651,6.93669,79.9146,301.64,0,0,0,1,5,0,3.917991,1,0.003604
3,10.500000,598.000000,271.000000,15.663800,68.000000,6.92570,79.8895,6.92748,79.8971,82.30,0,0,0,2,5,0,0.535588,1,0.000896
4,13.719651,1702.858077,629.074231,32.057666,112.466832,6.87441,79.8615,6.84478,79.9290,358.39,1,1,0,3,5,0,5.062797,1,0.002973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17171,10.500000,838.000000,93.000000,5.421900,451.000000,7.29073,80.6367,7.28891,80.6557,198.26,0,0,0,22,5,0,1.308220,1,0.001561
17172,10.500000,2151.000000,428.000000,0.000000,39.000000,6.90569,79.8516,6.95089,79.9389,581.23,0,0,0,23,5,0,6.753297,1,0.003140
17173,10.500000,263.000000,9.000000,0.000000,110.000000,7.09210,79.9000,7.10135,79.9017,76.20,0,0,0,23,5,0,0.649655,1,0.002470
17174,10.500000,858.000000,115.000000,0.000000,317.000000,6.94540,79.8768,6.93574,79.9010,133.31,0,0,0,23,5,0,1.788975,1,0.002085


In [32]:
# #We can choose to drop the last 6 features from in our new model to reduce dimensionality, and thus save training time and space

# X = X_train.drop(['additional_fare','pickup_day_of_week','pickup_month','pickup_day'], axis=1)

## CatBoost

In [33]:
# !pip install catboost

In [ ]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(iterations=100000, 
                           cat_features=categorical_var_indices, 
#                            task_type="GPU",
                           devices='0:1',
                          nan_mode='Max')
model.fit(X_train,
          y_train,
          verbose=True)




Learning rate set to 0.000462
0:	learn: 0.6925983	total: 129ms	remaining: 3h 35m 43s
1:	learn: 0.6920253	total: 201ms	remaining: 2h 47m 9s
2:	learn: 0.6914784	total: 261ms	remaining: 2h 25m
3:	learn: 0.6909991	total: 324ms	remaining: 2h 14m 47s
4:	learn: 0.6904994	total: 377ms	remaining: 2h 5m 29s
5:	learn: 0.6898997	total: 425ms	remaining: 1h 57m 55s
6:	learn: 0.6893295	total: 477ms	remaining: 1h 53m 27s
7:	learn: 0.6887686	total: 528ms	remaining: 1h 50m 3s
8:	learn: 0.6883073	total: 580ms	remaining: 1h 47m 19s
9:	learn: 0.6877902	total: 640ms	remaining: 1h 46m 34s
10:	learn: 0.6873289	total: 701ms	remaining: 1h 46m 8s
11:	learn: 0.6867599	total: 763ms	remaining: 1h 45m 58s
12:	learn: 0.6862251	total: 831ms	remaining: 1h 46m 33s
13:	learn: 0.6857321	total: 897ms	remaining: 1h 46m 49s
14:	learn: 0.6852565	total: 955ms	remaining: 1h 46m 4s
15:	learn: 0.6847598	total: 1.02s	remaining: 1h 45m 56s
16:	learn: 0.6842076	total: 1.08s	remaining: 1h 46m 6s
17:	learn: 0.6836784	total: 1.15s	rema

150:	learn: 0.6199780	total: 8.16s	remaining: 1h 29m 59s
151:	learn: 0.6194889	total: 8.22s	remaining: 1h 30m
152:	learn: 0.6190830	total: 8.27s	remaining: 1h 29m 59s
153:	learn: 0.6185946	total: 8.34s	remaining: 1h 30m 6s
154:	learn: 0.6181943	total: 8.41s	remaining: 1h 30m 19s
155:	learn: 0.6177660	total: 8.47s	remaining: 1h 30m 21s
156:	learn: 0.6173849	total: 8.49s	remaining: 1h 30m
157:	learn: 0.6169861	total: 8.55s	remaining: 1h 30m
158:	learn: 0.6165217	total: 8.6s	remaining: 1h 30m 1s
159:	learn: 0.6160485	total: 8.65s	remaining: 1h 29m 56s
160:	learn: 0.6156042	total: 8.69s	remaining: 1h 29m 47s
161:	learn: 0.6152194	total: 8.73s	remaining: 1h 29m 42s
162:	learn: 0.6147950	total: 8.78s	remaining: 1h 29m 40s
163:	learn: 0.6143392	total: 8.83s	remaining: 1h 29m 38s
164:	learn: 0.6139255	total: 8.89s	remaining: 1h 29m 36s
165:	learn: 0.6134852	total: 8.94s	remaining: 1h 29m 33s
166:	learn: 0.6130568	total: 8.99s	remaining: 1h 29m 36s
167:	learn: 0.6126005	total: 9.07s	remaining: 

298:	learn: 0.5610300	total: 15.6s	remaining: 1h 26m 48s
299:	learn: 0.5607237	total: 15.7s	remaining: 1h 26m 46s
300:	learn: 0.5603680	total: 15.7s	remaining: 1h 26m 47s
301:	learn: 0.5599773	total: 15.8s	remaining: 1h 26m 53s
302:	learn: 0.5596419	total: 15.9s	remaining: 1h 26m 58s
303:	learn: 0.5592116	total: 15.9s	remaining: 1h 27m 1s
304:	learn: 0.5588292	total: 16s	remaining: 1h 27m 1s
305:	learn: 0.5584514	total: 16s	remaining: 1h 27m
306:	learn: 0.5581097	total: 16.1s	remaining: 1h 27m
307:	learn: 0.5577024	total: 16.1s	remaining: 1h 27m 2s
308:	learn: 0.5574061	total: 16.2s	remaining: 1h 26m 54s
309:	learn: 0.5570723	total: 16.2s	remaining: 1h 26m 51s
310:	learn: 0.5567106	total: 16.3s	remaining: 1h 26m 49s
311:	learn: 0.5564018	total: 16.3s	remaining: 1h 26m 42s
312:	learn: 0.5559773	total: 16.3s	remaining: 1h 26m 40s
313:	learn: 0.5556413	total: 16.4s	remaining: 1h 26m 35s
314:	learn: 0.5552854	total: 16.4s	remaining: 1h 26m 35s
315:	learn: 0.5549619	total: 16.5s	remaining: 

448:	learn: 0.5108067	total: 22.8s	remaining: 1h 24m 15s
449:	learn: 0.5104918	total: 22.8s	remaining: 1h 24m 14s
450:	learn: 0.5101718	total: 22.9s	remaining: 1h 24m 13s
451:	learn: 0.5098717	total: 22.9s	remaining: 1h 24m 10s
452:	learn: 0.5095166	total: 23s	remaining: 1h 24m 7s
453:	learn: 0.5092640	total: 23s	remaining: 1h 24m 1s
454:	learn: 0.5089045	total: 23s	remaining: 1h 23m 58s
455:	learn: 0.5086101	total: 23.1s	remaining: 1h 23m 56s
456:	learn: 0.5082600	total: 23.1s	remaining: 1h 23m 56s
457:	learn: 0.5080087	total: 23.1s	remaining: 1h 23m 51s
458:	learn: 0.5076700	total: 23.2s	remaining: 1h 23m 51s
459:	learn: 0.5073619	total: 23.3s	remaining: 1h 23m 52s
460:	learn: 0.5071041	total: 23.3s	remaining: 1h 23m 52s
461:	learn: 0.5068200	total: 23.4s	remaining: 1h 23m 53s
462:	learn: 0.5065138	total: 23.4s	remaining: 1h 23m 54s
463:	learn: 0.5061527	total: 23.5s	remaining: 1h 23m 55s
464:	learn: 0.5059234	total: 23.5s	remaining: 1h 23m 48s
465:	learn: 0.5056091	total: 23.6s	rema

597:	learn: 0.4689987	total: 30s	remaining: 1h 23m 10s
598:	learn: 0.4687150	total: 30.1s	remaining: 1h 23m 10s
599:	learn: 0.4684608	total: 30.1s	remaining: 1h 23m 11s
600:	learn: 0.4682243	total: 30.2s	remaining: 1h 23m 10s
601:	learn: 0.4679721	total: 30.2s	remaining: 1h 23m 13s
602:	learn: 0.4676599	total: 30.3s	remaining: 1h 23m 15s
603:	learn: 0.4674054	total: 30.4s	remaining: 1h 23m 16s
604:	learn: 0.4671932	total: 30.4s	remaining: 1h 23m 15s
605:	learn: 0.4669476	total: 30.5s	remaining: 1h 23m 16s
606:	learn: 0.4666722	total: 30.5s	remaining: 1h 23m 16s
607:	learn: 0.4663714	total: 30.6s	remaining: 1h 23m 18s
608:	learn: 0.4661098	total: 30.6s	remaining: 1h 23m 19s
609:	learn: 0.4658120	total: 30.7s	remaining: 1h 23m 19s
610:	learn: 0.4655574	total: 30.7s	remaining: 1h 23m 18s
611:	learn: 0.4653262	total: 30.8s	remaining: 1h 23m 18s
612:	learn: 0.4650597	total: 30.8s	remaining: 1h 23m 18s
613:	learn: 0.4647740	total: 30.9s	remaining: 1h 23m 18s
614:	learn: 0.4645677	total: 30.9

...

24722:	learn: 0.1340390	total: 20m 52s	remaining: 1h 3m 32s
24723:	learn: 0.1340390	total: 20m 52s	remaining: 1h 3m 32s
24724:	learn: 0.1340359	total: 20m 52s	remaining: 1h 3m 31s
24725:	learn: 0.1340342	total: 20m 52s	remaining: 1h 3m 31s
24726:	learn: 0.1340324	total: 20m 52s	remaining: 1h 3m 31s
24727:	learn: 0.1340306	total: 20m 52s	remaining: 1h 3m 31s
24728:	learn: 0.1340294	total: 20m 52s	remaining: 1h 3m 31s
24729:	learn: 0.1340274	total: 20m 52s	remaining: 1h 3m 31s
24730:	learn: 0.1340255	total: 20m 52s	remaining: 1h 3m 31s
24731:	learn: 0.1340246	total: 20m 52s	remaining: 1h 3m 31s
24732:	learn: 0.1340238	total: 20m 52s	remaining: 1h 3m 31s
24733:	learn: 0.1340226	total: 20m 52s	remaining: 1h 3m 31s
24734:	learn: 0.1340214	total: 20m 52s	remaining: 1h 3m 31s
24735:	learn: 0.1340199	total: 20m 52s	remaining: 1h 3m 31s
24736:	learn: 0.1340179	total: 20m 52s	remaining: 1h 3m 31s
24737:	learn: 0.1340160	total: 20m 52s	remaining: 1h 3m 31s
24738:	learn: 0.1340144	total: 20m 52s	r

24860:	learn: 0.1338315	total: 20m 58s	remaining: 1h 3m 25s
24861:	learn: 0.1338298	total: 20m 59s	remaining: 1h 3m 25s
24862:	learn: 0.1338288	total: 20m 59s	remaining: 1h 3m 24s
24863:	learn: 0.1338275	total: 20m 59s	remaining: 1h 3m 24s
24864:	learn: 0.1338264	total: 20m 59s	remaining: 1h 3m 24s
24865:	learn: 0.1338254	total: 20m 59s	remaining: 1h 3m 24s
24866:	learn: 0.1338242	total: 20m 59s	remaining: 1h 3m 24s
24867:	learn: 0.1338242	total: 20m 59s	remaining: 1h 3m 24s
24868:	learn: 0.1338218	total: 20m 59s	remaining: 1h 3m 24s
24869:	learn: 0.1338209	total: 20m 59s	remaining: 1h 3m 24s
24870:	learn: 0.1338199	total: 20m 59s	remaining: 1h 3m 24s
24871:	learn: 0.1338189	total: 20m 59s	remaining: 1h 3m 24s
24872:	learn: 0.1338169	total: 20m 59s	remaining: 1h 3m 24s
24873:	learn: 0.1338162	total: 20m 59s	remaining: 1h 3m 24s
24874:	learn: 0.1338150	total: 20m 59s	remaining: 1h 3m 24s
24875:	learn: 0.1338142	total: 20m 59s	remaining: 1h 3m 24s
24876:	learn: 0.1338124	total: 20m 59s	r

25003:	learn: 0.1336404	total: 21m 5s	remaining: 1h 3m 16s
25004:	learn: 0.1336393	total: 21m 5s	remaining: 1h 3m 16s
25005:	learn: 0.1336362	total: 21m 5s	remaining: 1h 3m 16s
25006:	learn: 0.1336346	total: 21m 5s	remaining: 1h 3m 16s
25007:	learn: 0.1336339	total: 21m 5s	remaining: 1h 3m 16s
25008:	learn: 0.1336332	total: 21m 5s	remaining: 1h 3m 16s
25009:	learn: 0.1336309	total: 21m 6s	remaining: 1h 3m 15s
25010:	learn: 0.1336301	total: 21m 6s	remaining: 1h 3m 15s
25011:	learn: 0.1336290	total: 21m 6s	remaining: 1h 3m 15s
25012:	learn: 0.1336279	total: 21m 6s	remaining: 1h 3m 15s
25013:	learn: 0.1336259	total: 21m 6s	remaining: 1h 3m 15s
25014:	learn: 0.1336244	total: 21m 6s	remaining: 1h 3m 15s
25015:	learn: 0.1336230	total: 21m 6s	remaining: 1h 3m 15s
25016:	learn: 0.1336213	total: 21m 6s	remaining: 1h 3m 15s
25017:	learn: 0.1336201	total: 21m 6s	remaining: 1h 3m 15s
25018:	learn: 0.1336185	total: 21m 6s	remaining: 1h 3m 15s
25019:	learn: 0.1336169	total: 21m 6s	remaining: 1h 3m 1

25143:	learn: 0.1334563	total: 21m 12s	remaining: 1h 3m 8s
25144:	learn: 0.1334563	total: 21m 12s	remaining: 1h 3m 8s
25145:	learn: 0.1334557	total: 21m 12s	remaining: 1h 3m 8s
25146:	learn: 0.1334548	total: 21m 12s	remaining: 1h 3m 8s
25147:	learn: 0.1334532	total: 21m 12s	remaining: 1h 3m 8s
25148:	learn: 0.1334520	total: 21m 12s	remaining: 1h 3m 8s
25149:	learn: 0.1334492	total: 21m 12s	remaining: 1h 3m 8s
25150:	learn: 0.1334476	total: 21m 12s	remaining: 1h 3m 8s
25151:	learn: 0.1334473	total: 21m 12s	remaining: 1h 3m 8s
25152:	learn: 0.1334465	total: 21m 13s	remaining: 1h 3m 8s
25153:	learn: 0.1334441	total: 21m 13s	remaining: 1h 3m 8s
25154:	learn: 0.1334426	total: 21m 13s	remaining: 1h 3m 8s
25155:	learn: 0.1334423	total: 21m 13s	remaining: 1h 3m 7s
25156:	learn: 0.1334423	total: 21m 13s	remaining: 1h 3m 7s
25157:	learn: 0.1334412	total: 21m 13s	remaining: 1h 3m 7s
25158:	learn: 0.1334401	total: 21m 13s	remaining: 1h 3m 7s
25159:	learn: 0.1334385	total: 21m 13s	remaining: 1h 3m 

25286:	learn: 0.1332586	total: 21m 19s	remaining: 1h 3m
25287:	learn: 0.1332576	total: 21m 19s	remaining: 1h 3m
25288:	learn: 0.1332553	total: 21m 19s	remaining: 1h 3m
25289:	learn: 0.1332528	total: 21m 19s	remaining: 1h 3m
25290:	learn: 0.1332516	total: 21m 19s	remaining: 1h 3m
25291:	learn: 0.1332509	total: 21m 19s	remaining: 1h 3m
25292:	learn: 0.1332481	total: 21m 19s	remaining: 1h 3m
25293:	learn: 0.1332451	total: 21m 20s	remaining: 1h 3m
25294:	learn: 0.1332444	total: 21m 20s	remaining: 1h 3m
25295:	learn: 0.1332425	total: 21m 20s	remaining: 1h 3m
25296:	learn: 0.1332408	total: 21m 20s	remaining: 1h 3m
25297:	learn: 0.1332379	total: 21m 20s	remaining: 1h 3m
25298:	learn: 0.1332370	total: 21m 20s	remaining: 1h 3m
25299:	learn: 0.1332337	total: 21m 20s	remaining: 1h 3m
25300:	learn: 0.1332324	total: 21m 20s	remaining: 1h 3m
25301:	learn: 0.1332318	total: 21m 20s	remaining: 1h 3m
25302:	learn: 0.1332310	total: 21m 20s	remaining: 1h 3m
25303:	learn: 0.1332307	total: 21m 20s	remaining

#Test Model

In [157]:
predictions = model.predict(X_test)
predictions = [round(value) for value in predictions]
np.unique(predictions , return_counts=True) 

(array([0, 1]), array([ 261, 3175]))

In [158]:
accuracy = accuracy_score(y_test, predictions)
f1_macro = f1_score(y_test, predictions, average='macro')
f1_binary = f1_score(y_test, predictions, average='binary')
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("Macro F1: %.2f%%" % (f1_macro * 100.0))
print("Binary F1: %.2f%%" % (f1_binary * 100.0))

Accuracy: 97.56%
Macro F1: 92.19%
Binary F1: 98.66%


# Save Predictions

In [ ]:
model_new = model
model_new.fit(X,
          y,
          verbose=True)

In [ ]:
predictions_1 = model_new.predict(test_1)
# predictions_1 = [round(value) for value in y_pred_1]
np.unique(predictions_1 , return_counts=True)

In [ ]:
submission_df = pd.read_csv("predications_submission.csv", 
                            index_col="tripid")
submission_df['prediction'] = predictions_1

submission_df.to_csv('predictions/2_4_catboost_categorical_100000_epochs_null_fill_with_mean_cat_var_speed_istimeok.csv')

In [ ]:
#  y_pred = pd.DataFrame({'prediction':predictions_1})
#  y_pred["prediction"] = y_pred["prediction"].astype(int)
#  y_pred['prediction'] = y_pred['prediction'].apply(lambda x: 0 if x < 0.5 else 1)
#  y_pred.to_csv("predictions/predications_catboost_6_features_3_100000epochs.csv")
# y_pred.to_csv("/content/drive/My Drive/UNI/SEM7/CS4622_Machine_Learning/Project_1/Fair_Classification/predictions/predications_catboost_6_features_3_400000epochs.csv")